In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 12)

/home/user/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=4096,
    buffer_size=14000000,
    learning_rate=1e-4,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [4]:
model.set_parameters("trained_models/g1-reptile/sac")
# model.batch_size = 2048 * 5

In [ ]:
model.learn(
    total_timesteps=int(1e8), 
    log_interval=100,
    tb_log_name='reptile',
    reset_num_timesteps=False,
)

Logging to ./logs/sac/reptile_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 67       |
|    ep_rew_mean     | -326     |
| time/              |          |
|    episodes        | 100      |
|    fps             | 178      |
|    time_elapsed    | 45       |
|    total_timesteps | 8088     |
| train/             |          |
|    actor_loss      | 16.2     |
|    critic_loss     | 2.61     |
|    ent_coef        | 0.859    |
|    ent_coef_loss   | -2.32    |
|    learning_rate   | 0.0001   |
|    n_updates       | 665      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 69.7     |
|    ep_rew_mean     | -279     |
| time/              |          |
|    episodes        | 200      |
|    fps             | 177      |
|    time_elapsed    | 81       |
|    total_timesteps | 14460    |
| train/             |          |
|    actor_loss      | 9.63     |
|    critic_loss

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 803      |
|    ep_rew_mean     | 2.69e+03 |
| time/              |          |
|    episodes        | 1600     |
|    fps             | 172      |
|    time_elapsed    | 6017     |
|    total_timesteps | 1038612  |
| train/             |          |
|    actor_loss      | -263     |
|    critic_loss     | 1.2      |
|    ent_coef        | 0.0816   |
|    ent_coef_loss   | -0.203   |
|    learning_rate   | 0.0001   |
|    n_updates       | 86542    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.55e+03 |
|    ep_rew_mean     | 1.06e+04 |
| time/              |          |
|    episodes        | 1700     |
|    fps             | 172      |
|    time_elapsed    | 7553     |
|    total_timesteps | 1303644  |
| train/             |          |
|    actor_loss      | -277     |
|    critic_loss     | 0.865    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.51e+03 |
|    ep_rew_mean     | 1.11e+04 |
| time/              |          |
|    episodes        | 3100     |
|    fps             | 172      |
|    time_elapsed    | 16635    |
|    total_timesteps | 2871168  |
| train/             |          |
|    actor_loss      | -324     |
|    critic_loss     | 2.74     |
|    ent_coef        | 0.0936   |
|    ent_coef_loss   | 1.03     |
|    learning_rate   | 0.0001   |
|    n_updates       | 239255   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.39e+03 |
|    ep_rew_mean     | 1.06e+04 |
| time/              |          |
|    episodes        | 3200     |
|    fps             | 172      |
|    time_elapsed    | 18103    |
|    total_timesteps | 3124548  |
| train/             |          |
|    actor_loss      | -320     |
|    critic_loss     | 2.98     |
|    ent_coef 

In [ ]:
model.save("trained_models/g1-reptile/sac")